In [1]:
import requests
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import statistics

#### Creating the Earthquake dataset 1924 - present
Try not to run the code that builds the datasets very often in-case the website blocks us

In [2]:
dataframes = []
for page in range(1, 19):
    url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/earthquakes?itemsPerPage=200&minYear=1924&order=year%3Adesc&page={page}'
    response = requests.get(url, headers={"accept": "*/*"})
    df = pd.DataFrame(response.json()["items"])
    dataframes.append(df)

earthquake_df = pd.concat(dataframes, ignore_index=True)

#### Creating the Volcanoe dataset from 1924 to present

In [3]:
dataframes = []
for page in range(1, 4):
    url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/volcanoes?minYear=1924&order=year%3Adesc&page={page}'
    response = requests.get(url, headers={"accept": "*/*"})
    df = pd.DataFrame(response.json()["items"])
    dataframes.append(df)

volcanoe_df = pd.concat(dataframes, ignore_index=True)

#### Creating the Tsunami dataset 1924 - present 

In [4]:
dataframes = []
for page in range(1, 7):
    url = f'https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/tsunamis/events?itemsPerPage=200&minYear=1924&order=year%3Adesc&page={page}'
    response = requests.get(url, headers={"accept": "*/*"})
    df = pd.DataFrame(response.json()['items'])
    dataframes.append(df)

tsunami_df = pd.concat(dataframes, ignore_index=True)

#### Code to save the data frames in the core/ folder so we don't have to hit the api over and over again.
Once done, comment the code in this section and the code above

In [5]:
# earthquake_df.to_csv('core/earthquake_data.csv')
# volcanoe_df.to_csv('core/volcanoe_data.csv')
# tsunami_df.to_csv('core/tsunami_data.csv')

#### This reads the csv files and creates the dataframes we will want to use in our work

In [6]:
# earthquake_df = pd.read_csv('core/earthquake_data.csv')
# volcanoe_df = pd.read_csv('core/volcanoe_data.csv')
# tsunami_df = pd.read_csv('core/tsunami_data.csv')

#### This will show you the table headers

In [7]:
# display(earthquake_df.columns)
# display(volcanoe_df.columns)
# display(tsunami_df.columns)

In [8]:
earthquake_df['country'] = earthquake_df['country'].str.lower()
tsunami_df['country'] = tsunami_df['country'].str.lower()
volcanoe_df['country'] = volcanoe_df['country'].str.lower()

earthquake_data_filtered = earthquake_df[['country']].assign(disaster_type='earthquake')
tsunami_data_filtered = tsunami_df[['country']].assign(disaster_type='tsunami')
volcanoe_data_filtered = volcanoe_df[['country']].assign(disaster_type='volcano')
combined_data = pd.concat([earthquake_data_filtered, tsunami_data_filtered, volcanoe_data_filtered])

In [ ]:
disaster_counts_by_type = combined_data.groupby(['country', 'disaster_type']).size().unstack(fill_value=0)
disaster_counts_by_type['total_disasters'] = disaster_counts_by_type.sum(axis=1)
top_10_countries = disaster_counts_by_type.sort_values(by='total_disasters', ascending=False).head(20)
plt.figure(figsize=(10, 6))
top_10_countries[['earthquake', 'volcano', 'tsunami']].plot(kind='bar', stacked=True, color=['blue', 'green', 'red'], ax=plt.gca())
plt.title('Top 20 Countries with the Most Natural Disasters (Stacked by Type)')
plt.xlabel('Country')
plt.ylabel('Number of Disasters')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Disaster Type')
plt.tight_layout()
plt.show()

In [10]:
earthquake_counts = earthquake_df.groupby('country').size().reset_index(name='earthquake_count')
most_earthquakes = earthquake_counts.sort_values(by='earthquake_count', ascending=False).head(20)

volcanoe_counts = volcanoe_df.groupby('country').size().reset_index(name='volcanic_eruption_count')
most_volcanoes = volcanoe_counts.sort_values(by='volcanic_eruption_count', ascending=False).head(20)

tsunami_counts = tsunami_df.groupby('country').size().reset_index(name='tsunami_count')
most_tsunamis = tsunami_counts.sort_values(by='tsunami_count', ascending=False).head(20)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(most_earthquakes['country'], most_earthquakes['earthquake_count'], color='blue')
plt.title('Top 20 Countries with the Most Earthquakes')
plt.xlabel('Country')
plt.ylabel('Number of Earthquakes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(most_volcanoes['country'], most_volcanoes['volcanic_eruption_count'], color='green')
plt.title('Top 20 Countries with the Most Volcanic Eruptions')
plt.xlabel('Country')
plt.ylabel('Number of Volcanic Eruptions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(most_tsunamis['country'], most_tsunamis['tsunami_count'], color='red')
plt.title('Top 20 Countries with the Most Tsunamis')
plt.xlabel('Country')
plt.ylabel('Number of Tsunamis')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
earthquake_data = earthquake_df[['country', 'deaths', 'damageMillionsDollars', 'year']].rename(columns={'damageMillionsDollars': 'damage'})
tsunami_data = tsunami_df[['country', 'deaths', 'damageMillionsDollars', 'year']].rename(columns={'damageMillionsDollars': 'damage'})
volcano_data = volcanoe_df[['country', 'deaths', 'damageMillionsDollars', 'year']].rename(columns={'damageMillionsDollars': 'damage'})

combined_data = pd.concat([earthquake_data, tsunami_data, volcano_data], ignore_index=True)
combined_data['deaths'] = combined_data['deaths'].fillna(0)
combined_data['damage'] = combined_data['damage'].fillna(0)
combined_data = combined_data[combined_data['damage'] > 0]

def inflation_adjustment(year):
    return 1 + (18.36 - 0.1836 * (year - 1924))

combined_data['inflation_factor'] = combined_data['year'].apply(inflation_adjustment)
combined_data['damage_adjusted'] = combined_data['damage'] * combined_data['inflation_factor']


country_totals = combined_data.groupby('country').agg({
    'deaths': 'sum',
    'damage_adjusted': 'sum'
}).reset_index()

country_totals['total_impact'] = country_totals['deaths'] + country_totals['damage_adjusted']

highest_impact_countries = country_totals.sort_values(by='total_impact', ascending=False)
disaster_count_by_country = combined_data.groupby('country').size().reset_index(name='disaster_count')
most_disasters_countries = disaster_count_by_country.sort_values(by='disaster_count', ascending=False)

print("Top 10 Countries with the Highest Death and Destruction (inflation adjusted):")
print(highest_impact_countries.head(10))

print("\nTop 10 Countries with the Most Disasters:")
print(most_disasters_countries.head(10))

comparison = pd.merge(highest_impact_countries, most_disasters_countries, on='country', how='outer').fillna(0)
comparison = comparison[['country', 'total_impact', 'disaster_count']].sort_values(by='total_impact', ascending=False)

print("\nComparison of Top 10 Countries by Total Impact and Disaster Count:")
print(comparison.head(10))


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(comparison['disaster_count'], comparison['total_impact'], color='purple')
ax.set_xlabel('Disaster Count')
ax.set_ylabel('Total Impact (Deaths + Adjusted Damage)')
ax.set_title('Disaster Count vs Total Impact')
ax.grid(True)
plt.tight_layout()
plt.show()

corr, p_value = stats.pearsonr(comparison['disaster_count'], comparison['total_impact'])
print(f"Pearson Correlation Coefficient: {corr}")
print(f"P-value: {p_value}")

In [ ]:
#What is that outlier?
display(comparison[comparison['total_impact'] > 1*10**6])

#without the outlier
comparison_no_outlier = comparison[comparison['total_impact'] < 1*10**6]
corr, p_value = stats.pearsonr(comparison_no_outlier['disaster_count'], comparison_no_outlier['total_impact'])
print(f"Pearson Correlation Coefficient (without outlier): {corr}, P-value: {p_value}")

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(comparison_no_outlier['disaster_count'], comparison_no_outlier['total_impact'], color='green')
ax.set_xlabel('Disaster Count')
ax.set_ylabel('Total Impact (Deaths + Adjusted Damage)')
ax.set_title('Disaster Count vs Total Impact')
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
yearly_data = combined_data.groupby('year').agg({
    'deaths': 'sum',             
    'damage_adjusted': 'sum',    
    'year': 'size'               
}).rename(columns={'year': 'disaster_count'}).reset_index()

plt.figure(figsize=(10, 6))
plt.plot(yearly_data['year'], yearly_data['deaths'], marker='o', linestyle='-', color='red')
plt.title('Year vs. Total Deaths from Natural Disasters')
plt.xlabel('Year')
plt.ylabel('Total Deaths')
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(yearly_data['year'], yearly_data['damage_adjusted'], marker='o', linestyle='-', color='blue')
plt.title('Year vs. Total Inflation-Adjusted Damage from Natural Disasters')
plt.xlabel('Year')
plt.ylabel('Total Adjusted Damage (in today\'s dollars)')
plt.grid(True)
plt.show()

corr_deaths, p_value_deaths = stats.pearsonr(yearly_data['year'], yearly_data['deaths'])
corr_damage, p_value_damage = stats.pearsonr(yearly_data['year'], yearly_data['damage_adjusted'])

print(f"Pearson correlation (Year vs. Total Deaths): {corr_deaths}, p-value: {p_value_deaths}")
print(f"Pearson correlation (Year vs. Total Adjusted Damage): {corr_damage}, p-value: {p_value_damage}")

plt.figure(figsize=(10, 6))
plt.plot(yearly_data['year'], yearly_data['disaster_count'], marker='o', linestyle='-', color='green')
plt.title('Year vs. Disaster Counts')
plt.xlabel('Year')
plt.ylabel('Number of Disasters')
plt.grid(True)
plt.show()

corr_disaster_count, p_value_disaster_count = stats.pearsonr(yearly_data['year'], yearly_data['disaster_count'])
print(f"Pearson correlation (Year vs. Disaster Count): {corr_disaster_count}, p-value: {p_value_disaster_count}")

In [ ]:
earthquake_deaths_damage = earthquake_df[['id', 'year', 'deaths', 'damageMillionsDollars']]
display(earthquake_deaths_damage.head())

volcanoe_deaths_damage = volcanoe_df[['id', 'year', 'deaths', 'damageMillionsDollars']]
display(volcanoe_deaths_damage.head())

tsunami_deaths_damage = tsunami_df[['id', 'year', 'deaths', 'damageMillionsDollars']]
display(tsunami_deaths_damage.head())

In [ ]:
#ANOVA test
# H0: The means of the deaths for each type of disaster are equal
# H1: The means of the deaths for each type of disaster are not equal

e_deaths = earthquake_deaths_damage['deaths'].fillna(0)
v_deaths = volcanoe_deaths_damage['deaths'].fillna(0)
t_deaths = tsunami_deaths_damage['deaths'].fillna(0)


display(stats.f_oneway(e_deaths, v_deaths, t_deaths))

display(statistics.median(e_deaths), statistics.median(v_deaths), statistics.median(t_deaths))
sns.kdeplot(e_deaths, label='Earthquake Deaths')
#very right skewed

#Now for values > 1000
edeaths_small = e_deaths[e_deaths < 1000]
vdeaths_small = v_deaths[v_deaths < 1000]
tdeaths_small = t_deaths[t_deaths < 1000]

display(stats.f_oneway(edeaths_small, vdeaths_small, tdeaths_small))
display(statistics.median(edeaths_small), statistics.median(vdeaths_small), statistics.median(tdeaths_small))

In [ ]:
sns.kdeplot(edeaths_small, label='Earthquake Deaths < 1000', color='blue')
sns.kdeplot(vdeaths_small, label='Volcanoe Deaths < 1000', color='green')
sns.kdeplot(tdeaths_small, label='Tsunami Deaths < 1000', color='red')

In [ ]:
earthquake_deaths_damage.loc[:, 'damageMillionsDollars_adj'] = earthquake_deaths_damage['damageMillionsDollars'] * earthquake_deaths_damage['year'].apply(inflation_adjustment)
volcanoe_deaths_damage.loc[:, 'damageMillionsDollars_adj'] = volcanoe_deaths_damage['damageMillionsDollars'] * volcanoe_deaths_damage['year'].apply(inflation_adjustment)
tsunami_deaths_damage.loc[:, 'damageMillionsDollars_adj'] = tsunami_deaths_damage['damageMillionsDollars'] * tsunami_deaths_damage['year'].apply(inflation_adjustment)

earthquake_deaths_damage_filtered = earthquake_deaths_damage[earthquake_deaths_damage['damageMillionsDollars_adj'] > 1]
volcanoe_deaths_damage_filtered = volcanoe_deaths_damage[volcanoe_deaths_damage['damageMillionsDollars_adj'] > 1]
tsunami_deaths_damage_filtered = tsunami_deaths_damage[tsunami_deaths_damage['damageMillionsDollars_adj'] > 1]

plt.figure(figsize=(10, 6))
sns.kdeplot(earthquake_deaths_damage_filtered['damageMillionsDollars_adj'], label='Earthquake Damages (Adjusted)', color='red')
sns.kdeplot(volcanoe_deaths_damage_filtered['damageMillionsDollars_adj'], label='Volcano Damages (Adjusted)', color='blue')
sns.kdeplot(tsunami_deaths_damage_filtered['damageMillionsDollars_adj'], label='Tsunami Damages (Adjusted)', color='green')
plt.title('KDE of Inflation-Adjusted Damages by Disaster Type')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(earthquake_deaths_damage_filtered['damageMillionsDollars_adj'], label='Earthquake Damages (Adjusted)', color='red', bins=30, kde=False)

plt.title('Earthquake Damages (Adjusted)', fontsize=14)
plt.xlabel('Inflation-Adjusted Damages (Millions of Dollars)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(volcanoe_deaths_damage_filtered['damageMillionsDollars_adj'], label='Volcano Damages (Adjusted)', color='blue', bins=30, kde=False)

plt.title('Volcano Damages (Adjusted)', fontsize=14)
plt.xlabel('Inflation-Adjusted Damages (Millions of Dollars)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(tsunami_deaths_damage_filtered['damageMillionsDollars_adj'], label='Tsunami Damages (Adjusted)', color='green', bins=30, kde=False)
plt.title('Tsunami Damages (Adjusted)', fontsize=14)
plt.xlabel('Inflation-Adjusted Damages (Millions of Dollars)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
top_earthquake_damages = earthquake_deaths_damage_filtered.sort_values(by='damageMillionsDollars_adj', ascending=False).head(10)
print("Top 10 Earthquake Damages (Adjusted):")
print(top_earthquake_damages[['damageMillionsDollars_adj', 'year']])

top_volcano_damages = volcanoe_deaths_damage_filtered.sort_values(by='damageMillionsDollars_adj', ascending=False).head(10)
print("Top 10 Volcano Damages (Adjusted):")
print(top_volcano_damages[['damageMillionsDollars_adj', 'year']])

top_tsunami_damages = tsunami_deaths_damage_filtered.sort_values(by='damageMillionsDollars_adj', ascending=False).head(10)
print("Top 10 Tsunami Damages (Adjusted):")
print(top_tsunami_damages[['damageMillionsDollars_adj', 'year']])

In [ ]:
e_damage = earthquake_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)
v_damage = volcanoe_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)
t_damage = tsunami_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)

e_damage = earthquake_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)
v_damage = volcanoe_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)
t_damage = tsunami_deaths_damage_filtered['damageMillionsDollars_adj'].fillna(0)


display(stats.f_oneway(e_damage, v_damage, t_damage))

display(statistics.median(e_damage), statistics.median(v_damage), statistics.median(t_damage))
sns.histplot(e_damage)
sns.histplot(v_damage)
sns.histplot(t_damage)
#very right skewed

# #Now for values > 1000
# edeaths_small = e_deaths[e_deaths < 1000]
# vdeaths_small = v_deaths[v_deaths < 1000]
# tdeaths_small = t_deaths[t_deaths < 1000]

# display(stats.f_oneway(edeaths_small, vdeaths_small, tdeaths_small))
# display(statistics.median(edeaths_small), statistics.median(vdeaths_small), statistics.median(tdeaths_small))